In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import mean_squared_error, mean_absolute_error, confusion_matrix
import os
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
#나눔 고딕 적용
import matplotlib.font_manager
font_list = matplotlib.font_manager.findSystemFonts(fontpaths=None, fontext='ttf')
[matplotlib.font_manager.FontProperties(fname=font).get_name() for font in font_list if 'Nanum' in font]
\
import matplotlib.pyplot as plt
plt.rc('font', family='NanumGothicCoding')
\
import matplotlib as mpl
mpl.rcParams['axes.unicode_minus'] = False

matplotlib.rc("axes", unicode_minus = False)
from sklearn.model_selection import train_test_split
# export_graphviz: 나무 구조 생성 및 저장
from sklearn.tree import export_graphviz
import graphviz
from sklearn.model_selection import train_test_split, LeavePOut
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
from sklearn.model_selection import KFold
import itertools
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [2]:
def measure(clf, x_test, y_test, target_name=None):
    y_pred = clf.predict(x_test)
    report = classification_report(y_test, y_pred, target_names=target_name)
    print("분류 보고서:\n")
    print(report)
    cm = confusion_matrix(y_test, y_pred)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=target_name)
    disp.plot(cmap=plt.cm.Blues)
    plt.show()

In [3]:
df_raw = pd.read_csv("../data/Customer_data.csv", encoding="euc-kr")

In [8]:
save_path = "/home/piai/빅데이터프로젝트/data"
file_name1 = "../data/preprocessed_v1.0.csv"
file_name2 = "../data/Charge_data.csv"
df1 = pd.read_csv(os.path.join(save_path, file_name1), encoding="euc-kr")
df2 = pd.read_csv(os.path.join(save_path, file_name2), encoding="euc-kr")

In [9]:
# 데이터 로드
customer_data = df_raw

billing_data = df2

In [10]:
import numpy as np
import pandas as pd

def nearest_first_day_of_month(date):
    # 주어진 날짜의 월과 연도를 추출합니다.
    month = date.month
    year = date.year
    
    # 해당 월의 1일을 만듭니다.
    first_day_of_month = pd.Timestamp(year, month, 1)
    
    # 주어진 날짜가 해당 월의 1일에 가까운지 확인합니다.
    if date - first_day_of_month <= np.timedelta64(15, 'D'):
        return first_day_of_month
    else:
        # 다음 달의 1일을 반환합니다.
        if month == 12:
            return pd.Timestamp(year + 1, 1, 1)
        else:
            return pd.Timestamp(year, month + 1, 1)

In [13]:
billing_data['과금일'] = pd.to_datetime(billing_data['과금일'])
billing_data['과금일'] = billing_data['과금일'].apply(nearest_first_day_of_month)

In [21]:
billing_data['과금일'] = pd.to_datetime(billing_data['과금일'])

# Calculate billing statistics
billing_stats = billing_data.groupby('고객ID').agg(
    시작일=('과금일', 'min'),
    종료일=('과금일', 'max'),
    총과금액=('과금액', 'sum')
).reset_index()

# Calculate number of months between 시작일 and 종료일
billing_stats['개월수'] = (billing_stats['종료일'].dt.year - billing_stats['시작일'].dt.year) * 12 + \
                          (billing_stats['종료일'].dt.month - billing_stats['시작일'].dt.month) + 1

# Merge the customer data with billing statistics
merged_data = pd.merge(customer_data, billing_stats, on='고객ID', how='left')



In [22]:
merged_data.columns

Index(['고객ID', '성별', '연령', '결혼여부', '부양자유무', '부양자수', '추천횟수', '영수증발급여부', '과금방식',
       '보안서비스', '백업서비스', '기술지원서비스', '데이터무제한', '데이터사용량', '시작일_x', '종료일_x',
       '고객이탈여부', '이탈가능점수', '이탈유형', '이탈사유', 'LTV', '만족도', '데이터사용료', '로밍사용료',
       '총로밍요금', '시작일_y', '종료일_y', '총과금액', '개월수'],
      dtype='object')

In [24]:
merged_data.columns = ['고객ID', '성별', '연령', '결혼여부', '부양자유무', '부양자수', '추천횟수', '영수증발급여부', '과금방식',
       '보안서비스', '백업서비스', '기술지원서비스', '데이터무제한', '데이터사용량', '서비스시작일', '서비스종료일',
       '고객이탈여부', '이탈가능점수', '이탈유형', '이탈사유', 'LTV', '만족도', '데이터사용료', '로밍사용료',
       '총로밍요금', '납부시작일', '납부종료일', '총과금액', '개월수']

In [26]:
merged_data["월당객단가"] = merged_data["총과금액"] / merged_data["개월수"]

In [27]:
merged_data.columns

Index(['고객ID', '성별', '연령', '결혼여부', '부양자유무', '부양자수', '추천횟수', '영수증발급여부', '과금방식',
       '보안서비스', '백업서비스', '기술지원서비스', '데이터무제한', '데이터사용량', '서비스시작일', '서비스종료일',
       '고객이탈여부', '이탈가능점수', '이탈유형', '이탈사유', 'LTV', '만족도', '데이터사용료', '로밍사용료',
       '총로밍요금', '납부시작일', '납부종료일', '총과금액', '개월수', '월당객단가'],
      dtype='object')

In [28]:
import seaborn as sns